In [72]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
import time

In [1]:
import pandas as pd

In [3]:
train_bo2 = pd.read_pickle('/home/kai/data/shiyi/data/home_credit/train_bo2.pkl')
test_bo2 = pd.read_pickle('/home/kai/data/shiyi/data/home_credit/test_bo2.pkl')

train_bo2.SK_ID_CURR = train_bo2.SK_ID_CURR.astype(int)
test_bo2.SK_ID_CURR = test_bo2.SK_ID_CURR.astype(int)

In [131]:
cat_cols = []
for col in tqdm(test_bo2.columns):
    n = train_bo2[col].nunique()
    if n<10:
        cat_cols.append(col)

100%|██████████| 2678/2678 [00:07<00:00, 336.82it/s]


In [134]:
cat_cols.remove('TARGET')                                                                                

In [135]:
len(cat_cols)

199

In [136]:
import copy
train_count = copy.copy(train_bo2)
test_count = copy.copy(test_bo2)
train_count = train_count.fillna(-99999)
test_count = test_count.fillna(-99999)
train_count["test"]=0
test_count["test"]=1
temp_df = pd.concat([train_count,test_count],axis=0)
for col in tqdm(cat_cols):
    temp_df.loc[:,col]=temp_df.loc[:,col].astype('category')
train_count=temp_df[temp_df["test"]==0].iloc[:,:-1]
test_count=temp_df[temp_df["test"]==1].iloc[:,:-1]

100%|██████████| 199/199 [01:37<00:00,  2.04it/s]


In [137]:
for col in tqdm(cat_cols):
#     counts = train_count.iloc[:,i].value_counts()
    counts = train_count[col].value_counts()
    counts = counts.sort_index()
    counts = counts.fillna(0)
    counts += np.random.rand(len(counts))/1000
    train_count.loc[:,col].cat.categories = counts
    test_count.loc[:,col].cat.categories = counts

100%|██████████| 199/199 [00:00<00:00, 518.27it/s]


In [122]:
train_bo2[cat_cols[0]].value_counts()

0.0    264503
1.0      1292
2.0       106
3.0        45
4.0        26
5.0         9
6.0         8
9.0         2
8.0         1
Name: AMT_REQ_CREDIT_BUREAU_DAY, dtype: int64

In [123]:
train_count[cat_cols[0]].value_counts()

264503.000806    264503
41519.000378      41519
1292.000228        1292
106.000124          106
45.000926            45
26.000379            26
9.000728              9
8.000448              8
2.000487              2
1.000802              1
Name: AMT_REQ_CREDIT_BUREAU_DAY, dtype: int64

In [138]:
for col in cat_cols:
    train_count[col] = train_count[col].astype(float)
    test_count[col] = test_count[col].astype(float)

In [142]:
num_cols = [col for col in test_count.columns if col not in cat_cols]
train_count[num_cols] = train_count[num_cols].fillna(0)

In [143]:
train_p = train_count
X = train_p.drop(columns=['TARGET', 'SK_ID_CURR'])
y = train_p['TARGET']

In [144]:
from sklearn.linear_model import RidgeClassifier

from sklearn.model_selection import cross_val_score

clf = RidgeClassifier()

In [68]:
scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
scores = scores.mean()

0.7665596401369761

In [77]:
from tqdm import tqdm

In [150]:
# init_feats = fe_imp['feature'].tolist()[:3]
# init_score = cross_val_score(clf, X[init_feats], y, cv=5, scoring='roc_auc')
# init_score = init_score.mean()
# print('init auc: ', init_score)

featlist = fe_imp['feature'].tolist()[i:]
for i, feat in enumerate(featlist):
    scores = cross_val_score(clf, X[init_feats+[feat]], y, cv=5, scoring='roc_auc')
    auc = scores.mean()
    if auc > init_score:
        init_score = auc
        init_feats = init_feats+[feat]
        print('#{}/{} | YES! {} [{}] {:.6f}'.format(i, len(featlist), feat, len(init_feats), init_score))
    else:
        print('#{}/{} | NO!! {} [{}] {:.6f}'.format(i, len(featlist), feat, len(init_feats), init_score))
    time.sleep(1)

#0/1303 | NO!! prev_AMT_CREDIT_divide_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_1ST_VERSION_median [365] 0.737549
#1/1303 | NO!! prev_AMT_ANNUITY_divide_CNT_PAYMENT_sum [365] 0.737549
#2/1303 | NO!! inst_AMT_ONTIME_PAYMENT_min [365] 0.737549
#3/1303 | NO!! creditcard_AMT_BALANCE_divide_AMT_PAYMENT_CURRENT_median [365] 0.737549
#4/1303 | NO!! bureau_AMT_CREDIT_MAX_OVERDUE_divide_AMT_CREDIT_SUM_OVERDUE_median [365] 0.737549
#5/1303 | NO!! prev_AMT_GOODS_PRICE_divide_DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_LAST_DUE_max [365] 0.737549
#6/1303 | YES! bureau_AMT_CREDIT_SUM_divide_AMT_ANNUITY_mean [366] 0.737562
#7/1303 | YES! prev_CNT_PAYMENT_max [367] 0.737922
#8/1303 | NO!! prev_AMT_GOODS_PRICE_divide_DAYS_TERMINATION_min [367] 0.737922
#9/1303 | NO!! bureau_DAYS_CREDIT_ENDDATE_mean [367] 0.737922
#10/1303 | NO!! prev_DAYS_FIRST_DUE_minus_DAYS_LAST_DUE_sum [367] 0.737922
#11/1303 | YES! prev_DAYS_FIRST_DRAWING_minus_DAYS_TERMINATION_sum [368] 0.737926
#12/1303 | YES! bureau_onehot_STATUS_0_media

#102/1303 | NO!! prev_AMT_DOWN_PAYMENT_divide_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_sum [404] 0.740230
#103/1303 | NO!! inst_normalslope_AMT_LATE_PAYMENT_min_min [404] 0.740230
#104/1303 | YES! REGION_RATING_CLIENT_W_CITY_times_livingsum_MODE [405] 0.740238
#105/1303 | YES! creditcard_AMT_PAYMENT_CURRENT_divide_CNT_DRAWINGS_CURRENT_median [406] 0.740372
#106/1303 | YES! inst_LATE_DAYS_SIGN_sum [407] 0.740399
#107/1303 | YES! inst_DAYS_INSTALMENT_max [408] 0.740400
#108/1303 | YES! bureau_onehot_CREDIT_TYPE_Credit card [409] 0.740474
#109/1303 | NO!! prev_AMT_ANNUITY_divide_AMT_DOWN_PAYMENT_sum [409] 0.740474
#110/1303 | YES! prev_DAYS_LAST_DUE_1ST_VERSION_min [410] 0.740481
#111/1303 | NO!! bureau_MONTHS_BALANCE_max_mean [410] 0.740481
#112/1303 | YES! creditcard_AMT_BALANCE_divide_CNT_INSTALMENT_MATURE_CUM_mean [411] 0.740670
#113/1303 | NO!! prev_AMT_APPLICATION_divide_DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_LAST_DUE_min [411] 0.740670
#114/1303 | YES! WEEKDAY_APPR_PROCESS_START [412] 

#203/1303 | YES! def_divide_count_prev_min [443] 0.741479
#204/1303 | NO!! inst_AMT_INSTALMENT_min [443] 0.741479
#205/1303 | NO!! bureau_AMT_CREDIT_SUM_LIMIT_divide_CREDIT_DAY_OVERDUE_sum [443] 0.741479
#206/1303 | NO!! prev_count_SK_ID_CURR [443] 0.741479
#207/1303 | YES! bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_median [444] 0.741513
#208/1303 | YES! bureau_AMT_CREDIT_MAX_OVERDUE_divide_AMT_CREDIT_SUM_OVERDUE_std [445] 0.741543
#209/1303 | NO!! AMT_CREDIT_divide_DEF_30_CNT_SOCIAL_CIRCLE [445] 0.741543
#210/1303 | NO!! bureau_AMT_CREDIT_MAX_OVERDUE_divide_CREDIT_DAY_OVERDUE_mean [445] 0.741543
#211/1303 | NO!! AMT_CREDIT_divide_OBS_30_CNT_SOCIAL_CIRCLE [445] 0.741543
#212/1303 | YES! AMT_GOODS_PRICE_divide_AMT_REQ_CREDIT_BUREAU_MON [446] 0.741551
#213/1303 | NO!! bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_std [446] 0.741551
#214/1303 | NO!! bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_mean [446] 0.741551
#215/1303 | YES! bureau_AMT_CREDIT_MAX_OVERDUE_divide_DA

KeyboardInterrupt: 

In [ ]:
def temp(train):
    X = train.drop(columns=['TARGET', 'SK_ID_CURR'])
    y = train['TARGET']

    init_feats = fe_imp['feature'].tolist()[:3]
    init_score = cross_val_score(clf, X[init_feats], y, cv=5, scoring='roc_auc')
    init_score = init_score.mean()
    print('init auc: ', init_score)

    featlist = fe_imp['feature'].tolist()[3:]
    for i, feat in enumerate(featlist):
        scores = cross_val_score(clf, X[init_feats+[feat]], y, cv=5, scoring='roc_auc')
        auc = scores.mean()
        if auc > init_score:
            init_score = auc
            init_feats = init_feats+[feat]
            print('#{}/{} | YES! {} [{}] {:.6f}'.format(i, len(featlist), feat, len(init_feats), init_score))
        else:
            print('#{}/{} | NO!! {} [{}] {:.6f}'.format(i, len(featlist), feat, len(init_feats), init_score))
        time.sleep(1)
    return init_feats

In [ ]:
def temp(train):
    X = train.drop(columns=['TARGET', 'SK_ID_CURR'])
    y = train['TARGET']

    init_feats = ['EXT_SOURCE_2', 'EXT_SOURCE_3', 'EXT_SOURCE_1']#fe_imp['feature'].tolist()[:3]
    init_score = cross_val_score(clf, X[init_feats], y, cv=5, scoring='roc_auc')
    init_score = init_score.mean()
    print('init auc: ', init_score)

    featlist = list(train.columns)#fe_imp['feature'].tolist()[3:]
    for i, feat in enumerate(featlist):
        scores = cross_val_score(clf, X[init_feats+[feat]], y, cv=5, scoring='roc_auc')
        auc = scores.mean()
        if auc > init_score:
            init_score = auc
            init_feats = init_feats+[feat]
            print('#{}/{} | YES! {} [{}] {:.6f}'.format(i, len(featlist), feat, len(init_feats), init_score))
        else:
            print('#{}/{} | NO!! {} [{}] {:.6f}'.format(i, len(featlist), feat, len(init_feats), init_score))
        time.sleep(1)
    return init_feats

In [4]:
import xgboost as xgb

In [6]:
print_to_file = False

In [45]:
m = kfold_lightgbm(None, train_df.sample(frac=0.2), test_bo2, holdout_df, 5, 'xgbtest.csv', 'xgb_fe_img', True, False, colsample=0.1, lr=0.1, rounds=100, mode='xgb')

(52716, 2678) (48744, 2678) (43931, 2678)
MEAN: train(52716) vs holdout(43931):  0.08048789741255027 0.08108169629646492
[0]	val-auc:0.656363
[99]	val-auc:0.783984
Fold  1 [  100] AUC : ho: 0.765904 / te: 0.783984 / tr: -1.000000 (diff: 0.018080)
[0]	val-auc:0.657107
[99]	val-auc:0.771132
Fold  2 [  100] AUC : ho: 0.765245 / te: 0.771132 / tr: -1.000000 (diff: 0.005887)
[0]	val-auc:0.610611
[99]	val-auc:0.762706
Fold  3 [  100] AUC : ho: 0.763662 / te: 0.762706 / tr: -1.000000 (diff: 0.000956)
[0]	val-auc:0.638697
[99]	val-auc:0.759524
Fold  4 [  100] AUC : ho: 0.763602 / te: 0.759524 / tr: -1.000000 (diff: 0.004078)
[0]	val-auc:0.631884
[99]	val-auc:0.750993
Fold  5 [  100] AUC : ho: 0.763366 / te: 0.750993 / tr: -1.000000 (diff: 0.012373)
Full HO score 0.774838
FULL HO mean 0.764356, std 0.001022
FULL TE mean 0.765668, std 0.011200
FULL TR mean -1.000000, std 0.000000
FULL DIFF mean 0.008275, std 0.006161


In [47]:
fe_imp, _, _ = m

In [48]:
fe_imp.head()

,feature,importance
0,EXT_SOURCE_2,84
1,EXT_SOURCE_3,81
2,EXT_SOURCE_1,68
3,AMT_CREDIT_divide_AMT_ANNUITY,47
4,AMT_ANNUITY_divide_DAYS_EMPLOYED,38


In [39]:
train_df, holdout_df = train_test_split(train_bo2, test_size=1/7, random_state=4)
print(int(train_df['TARGET'].mean()*10000), int(holdout_df['TARGET'].mean()*10000))

806 810


In [44]:
from sklearn.model_selection import train_test_split

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, mode='sklearn', max_bin=255):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df['TARGET'].mean(), holdout['TARGET'].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df['TARGET'].notnull()]
        test_df = df[df['TARGET'].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
        
        
    # compute optimal rounds
    xgb_params = {
        'objective': 'binary:logistic',
#         'boosting': 'gbdt',
        'learning_rate': lr,
        'colsample_bytree': colsample,
        'eval_metric': 'auc'

#         'max_bin': max_bin,
#         'min_data_in_leaf': min_child_samples,
#         'num_leaves': num_leaves, 
#         'num_threads': 5, 
#         'max_depth': max_depth,
#         'min_child_weight': 39,
#         'feature_fraction': colsample,
#         'feature_fraction_seed': seed,
#         'bagging_fraction': subsample, 
#         'bagging_seed': seed,
#         'scale_pos_weight': 1,
#         'lambda_l1': 0.65, 
#         'lambda_l2': reg_lambda,
#         'metric': 'auc'
    }
    if rounds is None:
        print('computing optimal rounds...')
        xgb_train = xgb.DMatrix(train_df.drop(columns=['TARGET','SK_ID_CURR']), label=train_df['TARGET'])

        eval_hist = xgb.cv(xgb_params, xgb_train, nfold=num_folds, stratified=stratified, 
                           num_boost_round=50000, early_stopping_rounds=200, verbose_eval=verbose)
        best_round = len(eval_hist['test-auc-mean'])
#         best_round = len(eval_hist['auc-mean'])
        rounds = best_round
#         print('xgb cv: ', eval_hist['auc-mean'][-1], ' std: ', eval_hist['auc-stdv'][-1])
        print('xgb cv te: ', eval_hist['test-auc-mean'][-1], ' tr: ', eval_hist['train-auc-stdv'][-1])
        print('best rounds: ', rounds)

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        if mode=='sklearn':
            clf = xgbMClassifier(
                nthread=18,
                n_estimators=30000,
                learning_rate=lr,
                num_leaves=num_leaves,
                colsample_bytree=colsample, # 0.67
                subsample=subsample,
                subsample_freq=0, ## disable subsampling
                max_depth=max_depth,
                reg_alpha=0.65,
                reg_lambda=reg_lambda,
                min_split_gain=0.0222415,
                min_child_weight=39.3259775,
                min_child_samples=min_child_samples,
                silent=-1,
                verbose=-1, )
            if rounds is not None:
                clf.n_estimators = rounds
                clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                    eval_metric= 'auc', verbose=verbose)
                oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
                sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
                holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
            else:
                clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                    eval_metric= 'auc', verbose=verbose, early_stopping_rounds= 200)
                oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
                sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
                holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 

            train_score = clf.best_score_['training']['auc']
         
    
        else:
            xgb_train = xgb.DMatrix(train_x, label=train_y)
            xgb_val = xgb.DMatrix(valid_x, label=valid_y)
            xgb_val_x = xgb.DMatrix(valid_x)
            xgb_test = xgb.DMatrix(test_df[feats])
            xgb_holdout = xgb.DMatrix(holdout[feats])
#             xgb_model = xgb.train(xgb_params, train_set=xgb_train, valid_sets=[xgb_train, xgb_val], 
#                                   num_boost_round=rounds, verbose_eval=verbose)
            xgb_model = xgb.train(xgb_params, xgb_train, evals=[(xgb_val, 'val')], 
                                  num_boost_round=rounds, verbose_eval=verbose)
#             oof_preds[valid_idx] = xgb_model.predict(valid_x)
            oof_preds[valid_idx] = xgb_model.predict(xgb_val_x)
#             sub_preds += xgb_model.predict(test_df[feats]) / folds.n_splits
            sub_preds += xgb_model.predict(xgb_test) / folds.n_splits
#             holdout_preds = xgb_model.predict(holdout[feats])
            holdout_preds = xgb_model.predict(xgb_holdout)
            
            train_score = -1 #xgb_model.best_score['training']['auc'] 
        
        holdout_final_preds += holdout_preds / folds.n_splits
        score = roc_auc_score(valid_y, oof_preds[valid_idx])
        holdout_score = roc_auc_score(holdout['TARGET'], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds #if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)

        fold_importance_df = pd.DataFrame()
        if mode=='sklearn':
            fold_importance_df["feature"] = feats
            fold_importance_df["importance"] = clf.feature_importances_
        else:
#             fold_importance_df['feature'] = xgb_model.feature_name() 
#             fold_importance_df["importance"] = xgb_model.feature_importance() 
            fold_importance_df = pd.DataFrame.from_dict(m.get_fscore(), orient='index').reset_index().rename(columns={'index':'feature', 0:'importance'})
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        fold_importance_gain_df = pd.DataFrame()
        if mode=='sklearn':
            fold_importance_gain_df["feature"] = feats
            fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        else:
#             fold_importance_gain_df['feature'] = xgb_model.feature_name() 
#             fold_importance_gain_df["importance"] = xgb_model.feature_importance(importance_type='gain') 
            fold_importance_gain_df = pd.DataFrame.from_dict(m.get_fscore(), orient='index').reset_index().rename(columns={'index':'feature', 0:'importance'})
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)


#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % roc_auc_score(holdout['TARGET'], holdout_final_preds))
    print('FULL HO mean {:.6f}, std {:.6f}'.format(np.mean(holdout_scores), np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(np.mean(scores), np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(np.mean(train_scores), np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df, oof_preds 
        
#                 min_split_gain=0.0222415,

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))